<a href="https://colab.research.google.com/github/vixxypsycho/grade-d1/blob/main/Kaggle_Stage_D_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Imporing Required Libraries

import pandas as pd
import numpy as np
import os
import tarfile
import cv2
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from matplotlib.image import imread
from sklearn.metrics import fbeta_score
from tqdm import tqdm
import tensorflow as tf
from keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, InputLayer, Dense , Dropout , Flatten, Conv2D, MaxPooling2D , BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History, TensorBoard
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.backend import clear_session
import tensorflow_addons as tfa
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
# Specify the path to the tar file
tar_path = '/content/drive/MyDrive/Hamoye/planet-understanding-the-amazon-from-space/train-jpg.tar'

# Specify the directory to extract the images
extract_dir = 'data_path/'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Open the tar file
with tarfile.open(tar_path, 'r') as tar:
    # Extract all contents to the specified directory
    tar.extractall(path=extract_dir)

print(f"Images extracted to {extract_dir}")

Images extracted to data_path/


In [ ]:
# Specify the path to the tar file
tar_path2 = '/content/drive/MyDrive/Hamoye/planet-understanding-the-amazon-from-space/test-jpg.tar'

# Specify the directory to extract the images
extract_dir2 = 'data_path/'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir2, exist_ok=True)

# Open the tar file
with tarfile.open(tar_path2, 'r') as tar:
    # Extract all contents to the specified directory
    tar.extractall(path=extract_dir2)

print(f"Images extracted to {extract_dir2}")

Images extracted to data_path/


In [ ]:
# Specify the path to the tar file
tar_path3 = '/content/drive/MyDrive/Hamoye/planet-understanding-the-amazon-from-space/test-jpg-additional.tar'

# Specify the directory to extract the images
extract_dir3 = 'data_path/'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir3, exist_ok=True)

# Open the tar file
with tarfile.open(tar_path3, 'r') as tar:
    # Extract all contents to the specified directory
    tar.extractall(path=extract_dir3)

print(f"Images extracted to {extract_dir3}")

Images extracted to data_path/


In [ ]:
INPUT_SHAPE = (128, 128, 3) # Image Dimensions
BATCH_SIZE = 128
DROPOUT_RATE = 0.5
EPOCHS = 15
LR = 0.0001 # Learning Rate
REG_STRENGTH = 0.01 # Regularization Strength
NFOLDS = 2 # No of folds for cross validation
WORKERS = 4 # Multithreading no of threads
MAXQ = 10 # Max Queue size for multithreading
THRES = [0.2] * 17 # Threshold for truth value of label, applied on sigmoid output.

TRAIN_PATH = '/content/data_path/train-jpg'
TEST_PATH = '/content/data_path/test-jpg'
test_pat = '/content/data_path/test-jpg-additional'

TRAIN_CSV_PATH = '/content/drive/MyDrive/Hamoye/planet-understanding-the-amazon-from-space/train_v2.csv/train_v2.csv'
TEST_CSV_PATH = '/content/drive/MyDrive/Hamoye/planet-understanding-the-amazon-from-space/sample_submission_v2.csv/sample_submission_v2.csv'

In [ ]:
df_train = pd.read_csv(TRAIN_CSV_PATH)
df_test = pd.read_csv(TEST_CSV_PATH)


df_train['image_name'] = df_train['image_name'].astype(str) + '.jpg'
df_test['image_name'] = df_test['image_name'].astype(str) + '.jpg'


df_test['tags'] = df_test['tags'].apply(lambda x: x.split(' '))

print(df_train.head())
print('-------------------------------------------')
print(df_test.head())

X_train_files = np.array(df_train['image_name'].tolist())
X_train_files.reshape((X_train_files.shape[0], 1))

y_train = np.array(df_train['tags'].tolist())

    image_name                                       tags
0  train_0.jpg                               haze primary
1  train_1.jpg            agriculture clear primary water
2  train_2.jpg                              clear primary
3  train_3.jpg                              clear primary
4  train_4.jpg  agriculture clear habitation primary road
-------------------------------------------
   image_name                                        tags
0  test_0.jpg  [primary, clear, agriculture, road, water]
1  test_1.jpg  [primary, clear, agriculture, road, water]
2  test_2.jpg  [primary, clear, agriculture, road, water]
3  test_3.jpg  [primary, clear, agriculture, road, water]
4  test_4.jpg  [primary, clear, agriculture, road, water]


In [ ]:
labels = []

for tag in df_train['tags'].values:
    labels_in_tag = tag.split(' ')
    for label in labels_in_tag:
        if label not in labels:
            labels.append(label)

labels.sort()
print(labels)

['agriculture', 'artisinal_mine', 'bare_ground', 'blooming', 'blow_down', 'clear', 'cloudy', 'conventional_mine', 'cultivation', 'habitation', 'haze', 'partly_cloudy', 'primary', 'road', 'selective_logging', 'slash_burn', 'water']


In [ ]:
def create_model():
    model = Sequential()
    model.add(InputLayer(INPUT_SHAPE))
    model.add(VGG16(weights='imagenet', include_top=False))
    model.add(Flatten())
    model.add(Dense(17, activation='sigmoid'))
    return model

#clear_session()

model = create_model()
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 17)                139281    
                                                                 
Total params: 14853969 (56.66 MB)
Trainable params: 14853969 (56.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
def f2_score(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 5 * precision * recall / (4 * precision + recall)
    f_score = tf.where(tf.math.is_nan(f_score), tf.zeros_like(f_score), f_score)
    return tf.reduce_mean(f_score)

In [ ]:
num_fold = 0

y_test = []

folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=1).split(X_train_files, y_train)

for train_index, val_index in folds:
    X_train_files_fold = X_train_files[train_index]
    y_train_fold = y_train[train_index]
    X_val_files_fold = X_train_files[val_index]
    y_val_fold = np.array(y_train[val_index])

    train_df = pd.DataFrame(list(zip(X_train_files_fold, y_train_fold)), columns = ['image_name', 'tags'])
    val_df = pd.DataFrame(list(zip(X_val_files_fold, y_val_fold)), columns = ['image_name', 'tags'])

    train_df['tags'] = train_df['tags'].apply(lambda x: x.split(' '))
    val_df['tags'] = val_df['tags'].apply(lambda x: x.split(' '))

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True
    )

    train_generator = train_datagen.flow_from_dataframe(
        train_df,
        directory=TRAIN_PATH,
        x_col='image_name',
        y_col='tags',
        target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        classes=labels,
    )

    val_datagen = ImageDataGenerator(
        rescale=1./255
    )

    val_generator = val_datagen.flow_from_dataframe(
        val_df,
        directory=TRAIN_PATH,
        x_col='image_name',
        y_col='tags',
        target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        classes=labels,
    )

    test_datagen = ImageDataGenerator(
        rescale=1./255
    )

    test_generator = test_datagen.flow_from_dataframe(
        df_test[:40669],
        directory=TEST_PATH,
        x_col='image_name',
        y_col='tags',
        target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        classes=labels,
        shuffle=False,
    )


    test2_datagen = ImageDataGenerator(
        rescale=1./255
    )

    test2_generator = test2_datagen.flow_from_dataframe(
        df_test[40669:],
        directory=test_pat,
        x_col='image_name',
        y_col='tags',
        target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        classes=labels,
        shuffle=False,
    )

    model_path_of_fold = os.path.join('', 'weights_of_fold_' + str(num_fold) + '.h5')

    #clear_session()
    model = create_model()

    adam = Adam(learning_rate=LR)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[f2_score])

    callbacks = [
        ModelCheckpoint(model_path_of_fold, monitor='val_f2_score', save_best_only=True, mode='max'),
        ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, mode='min', min_lr=0.000001)
    ]

    model.fit_generator(train_generator, epochs=EPOCHS, validation_data=val_generator, callbacks=callbacks,
                       workers=WORKERS, use_multiprocessing=True, max_queue_size=MAXQ)

    model.load_weights(model_path_of_fold)

    p_test = model.predict_generator(test_generator, workers=WORKERS, use_multiprocessing=True, max_queue_size=MAXQ)
    p_test2 = model.predict_generator(test2_generator, workers=WORKERS, use_multiprocessing=True, max_queue_size=MAXQ)
    y_test.append(p_test)
    y_test.append(p_test2)

Found 20239 validated image filenames belonging to 17 classes.
Found 20240 validated image filenames belonging to 17 classes.
Found 40669 validated image filenames belonging to 17 classes.
Found 20522 validated image filenames belonging to 17 classes.
Epoch 1/15
159/159 [==============================] - 242s 1s/step - loss: 0.1730 - f2_score: 0.7784 - val_loss: 0.1276 - val_f2_score: 0.8459 - lr: 1.0000e-04
Epoch 2/15
159/159 [==============================] - 180s 1s/step - loss: 0.1278 - f2_score: 0.8466 - val_loss: 0.1175 - val_f2_score: 0.8807 - lr: 1.0000e-04
Epoch 3/15
159/159 [==============================] - 208s 1s/step - loss: 0.1163 - f2_score: 0.8626 - val_loss: 0.1068 - val_f2_score: 0.8795 - lr: 1.0000e-04
Epoch 4/15
159/159 [==============================] - 179s 1s/step - loss: 0.1116 - f2_score: 0.8692 - val_loss: 0.1087 - val_f2_score: 0.8906 - lr: 1.0000e-04
Epoch 5/15
159/159 [==============================] - 170s 1s/step - loss: 0.1066 - f2_score: 0.8760 - val_l

In [ ]:
re1 = np.array(y_test[0])
re2 = np.array(y_test[1])
re3 = np.array(y_test[2])
re4 = np.array(y_test[3])
result_test = re1 + re3
result_test /= NFOLDS
result_test = pd.DataFrame(result_test, columns = labels)

result_add_test = re2 + re4
result_add_test /= NFOLDS
result_add_test = pd.DataFrame(result_add_test, columns = labels)

In [ ]:
final = pd.concat([result_test, result_add_test], ignore_index=True)
final.reset_index(drop=True)
final.head()

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0.011696,0.000011,0.000145,0.062645,0.001265,0.996971,3.202226e-06,0.000002,0.008401,0.001266,0.000768,0.001582,0.999995,0.002361,0.004714,0.000083,0.005428
1,0.021752,0.000021,0.000339,0.048529,0.008381,0.996853,8.671107e-07,0.000007,0.026028,0.001417,0.000301,0.002712,0.999999,0.001492,0.008729,0.000246,0.003513
2,0.032158,0.000158,0.000852,0.000772,0.000686,0.000021,4.681208e-04,0.000044,0.014561,0.003098,0.000195,0.999790,0.999459,0.009388,0.004276,0.000273,0.049512
3,0.812154,0.000130,0.002971,0.008801,0.002480,0.929537,7.093317e-07,0.000016,0.796472,0.004665,0.000972,0.135018,0.999997,0.013507,0.006665,0.008125,0.030016
4,0.023421,0.000063,0.000697,0.000039,0.000030,0.001573,1.730966e-01,0.000004,0.004711,0.002124,0.002585,0.789563,0.851412,0.008228,0.000145,0.000011,0.029098


In [ ]:
re1 = np.array(y_test[0])
re1 = pd.DataFrame(re1, columns = labels)
print(re1.shape)
re2 = np.array(y_test[1])
re2 = pd.DataFrame(re2, columns = labels)
print(re2.shape)
re3 = np.array(y_test[2])
re3 = pd.DataFrame(re3, columns = labels)
print(re3.shape)
re4 = np.array(y_test[3])
re4 = pd.DataFrame(re4, columns = labels)
print(re4.shape)

(40669, 17)
(20522, 17)
(40669, 17)
(20522, 17)


In [ ]:
#result.to_csv('test.csv', index=False)
result_add_test.to_csv('test22.csv', index=False)
result_test.to_csv('test1.csv', index=False)

In [ ]:
final.to_csv('final_array.csv', index=False)

In [ ]:
final = pd.read_csv('/content/final_array.csv')
test1= pd.read_csv('/content/test1.csv')
test2 = pd.read_csv('/content/test22.csv')

In [ ]:
test_datagen = ImageDataGenerator(
            rescale=1./255
            )

test_generator = test_datagen.flow_from_dataframe(
    df_test[:40669],
    directory=TEST_PATH,
    x_col='image_name',
    y_col='tags',
    target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    classes=labels,
    shuffle=False,
    )


test2_datagen = ImageDataGenerator(
            rescale=1./255
            )

test2_generator = test2_datagen.flow_from_dataframe(
            df_test[40669:],
            directory=test_pat,
            x_col='image_name',
            y_col='tags',
            target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
            class_mode='categorical',
            batch_size=BATCH_SIZE,
            classes=labels,
            shuffle=False,
            )

Found 40669 validated image filenames belonging to 17 classes.
Found 20522 validated image filenames belonging to 17 classes.


In [ ]:
test1.shape

(40669, 17)

In [ ]:
test2.shape

(20522, 17)

In [ ]:
len(test2_generator.filenames)

20522

In [ ]:
pred_tags = test1.apply(lambda x: ' '.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this
result1 = pd.DataFrame({'image_name': test_generator.filenames, 'tags': pred_tags})
result1.head()

,image_name,tags
0,test_0.jpg,clear primary
1,test_1.jpg,clear primary
2,test_2.jpg,partly_cloudy primary
3,test_3.jpg,agriculture clear cultivation primary
4,test_4.jpg,partly_cloudy primary


In [ ]:
pred_tags2 = test2.apply(lambda x: ' '.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this
result2 = pd.DataFrame({'image_name': test2_generator.filenames, 'tags': pred_tags2})
result2.head()

,image_name,tags
0,file_0.jpg,clear primary
1,file_1.jpg,agriculture partly_cloudy primary
2,file_10.jpg,agriculture primary road water
3,file_100.jpg,clear primary water
4,file_1000.jpg,clear primary


In [ ]:
final_test = pd.concat([result1, result2])
final = final_test.reset_index().drop('index', axis =1)
final.head()

,image_name,tags
0,test_0.jpg,clear primary
1,test_1.jpg,clear primary
2,test_2.jpg,partly_cloudy primary
3,test_3.jpg,agriculture clear cultivation primary
4,test_4.jpg,partly_cloudy primary


In [ ]:
final.shape

(61191, 2)

In [ ]:
#we need to remove the .jpg extension from the image_name of the
# final result because from the sample submission the
#extension was not there, we added it for easy manipulation of the data.
final['image_name'] = final['image_name'].apply(lambda x: x[:-4])
final.head()

,image_name,tags
0,test_0,clear primary
1,test_1,clear primary
2,test_2,partly_cloudy primary
3,test_3,agriculture clear cultivation primary
4,test_4,partly_cloudy primary


In [ ]:
final.to_csv('final_submit.csv', index=False)